### Day 18: RAM Run

Link: https://adventofcode.com/2024/day/18#part2

Given the grid area of 71x71 and the input size of 3,450, it's technically possible to try falling byte positions one by one until we find the one that blocks the path, corresponding to `O(n * m)`, where `n` is the grid area and `m` is the input size. However, we can use binary search instead since once a path is blocked by a falling byte, all subsequent positions would also keep the path blocked. This approach reduces time complexity to `O(n log m)`.

In [7]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
import typing as t
from collections import deque
from dataclasses import dataclass


@dataclass
class Position:
    x: int
    y: int

    def __eq__(self, other: t.Any) -> bool:
        if not isinstance(other, Position):
            return False

        return (self.x, self.y) == (other.x, other.y)


@dataclass
class State:
    position: Position
    step_count: int


bytes_positions: list[Position] = []


for line in lines:
    x, y = map(int, line.strip().split(","))
    bytes_positions.append(Position(x=x, y=y))


grid_height = 71
grid_width = 71
grid = [["."] * grid_width for _ in range(grid_height)]


def reset_grid(bytes_falling: int) -> None:
    global grid
    grid = [["."] * grid_width for _ in range(grid_height)]

    for byte_idx in range(bytes_falling):
        byte_position = bytes_positions[byte_idx]
        grid[byte_position.y][byte_position.x] = "#"


target_position = Position(x=grid_width - 1, y=grid_height - 1)
actions = [
    (-1, 0),  # Up
    (1, 0),  # Down
    (0, -1),  # Left
    (0, 1),  # Right
]


def is_within_grid(position: Position) -> bool:
    return 0 <= position.y < grid_height and 0 <= position.x < grid_width


def is_valid_position(position: Position) -> bool:
    return is_within_grid(position) and grid[new_position.y][new_position.x] != "#"


left, right = 0, len(bytes_positions) - 1


while left < right:
    mid = (left + right) // 2
    reset_grid(mid)
    to_visit = deque([State(position=Position(x=0, y=0), step_count=0)])
    visited: set[tuple[int, int]] = set()
    is_blocked = False

    while to_visit:
        state = to_visit.popleft()

        if (state.position.x, state.position.y) in visited:
            continue

        visited.add((state.position.x, state.position.y))

        if state.position == target_position:
            break

        for add_row, add_column in actions:
            new_position = Position(
                x=state.position.x + add_column,
                y=state.position.y + add_row,
            )

            if is_valid_position(new_position):
                to_visit.append(State(position=new_position, step_count=state.step_count + 1))
    else:
        is_blocked = True

    if is_blocked:
        right = mid - 1
    else:
        left = mid + 1


position = bytes_positions[left]
print(f"{position.x},{position.y}")